In [1]:
import pandas as pd
import numpy as np
import requests
import json
import re



In [19]:
def managed(expense_ratio):
    if expense_ratio < 0.007:
        return 'passive'
    else:
        return 'active'

    
def get_price(ticker):

    url = 'https://finance.yahoo.com/quote/{}/history?'.format(ticker)
    params = {'interval':'1d', 
              'filter':'history', 
              'frequency':'1d'}
    
    r = requests.get(url, params=params)

    try:
        txt = re.search(r'root\.App\.main = (.*?);\n}\(this\)\);', r.text, re.DOTALL).group(1)
        jsn = json.loads(txt)
        prcs = pd.DataFrame(jsn['context']['dispatcher']['stores']['HistoricalPriceStore']['prices'])
        prcs['date'] = pd.to_datetime(prcs['date'], unit='s')
        prcs['date'] = pd.DatetimeIndex(prcs['date']).normalize()
        prcs = prcs[['open', 'high', 'low', 'close', 'adjclose', 'volume']]
        prcs = prcs.dropna(how='all').sort_index()
    except:
        prcs = pd.DataFrame(columns=['open', 'high', 'low', 'close', 
                                     'adjclose', 'volume'])
    
    return prcs

# takes in dictionary where key = Fund Ticker and value is a dataframe with price data

def df_date_range(pricing_data):
    start = '01/01/2015'
    end = '01/01/2015'

    for k in pricing_data:
        dfs = pricing_data.get(k)
        if dfs['date'].min() < start: start = dfs['date'].min() 
        if dfs['date'].max() > end: end = dfs['date'].max() 
        index = pd.date_range(start, end)
        return pd.DataFrame(index)



In [3]:
data = pd.read_csv('crsp.csv')

/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (45,47,59) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (45,47,59) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
data.shape

(458503, 70)

(458503, 70)

In [5]:
data.columns

Index(['summary_period2', 'crsp_fundno', 'caldt', 'nav_latest',
       'nav_latest_dt', 'tna_latest', 'tna_latest_dt', 'yield', 'div_ytd',
       'cap_gains_ytd', 'nav_52w_h', 'nav_52w_h_dt', 'nav_52w_l',
       'nav_52w_l_dt', 'unrealized_app_dep', 'unrealized_app_dt', 'asset_dt',
       'per_com', 'per_pref', 'per_conv', 'per_corp', 'per_muni', 'per_govt',
       'per_oth', 'per_cash', 'per_bond', 'per_abs', 'per_mbs', 'per_eq_oth',
       'per_fi_oth', 'maturity', 'maturity_dt', 'cusip8', 'crsp_portno',
       'crsp_cl_grp', 'fund_name', 'ticker', 'ncusip', 'mgmt_name', 'mgmt_cd',
       'mgr_name', 'open_to_inv', 'retail_fund', 'inst_fund', 'm_fund',
       'index_fund_flag', 'vau_fund', 'et_flag', 'delist_cd', 'end_dt',
       'dead_flag', 'merge_fundno', 'actual_12b1', 'max_12b1', 'exp_ratio',
       'mgmt_fee', 'turn_ratio', 'fiscal_yearend', 'crsp_obj_cd', 'si_obj_cd',
       'accrual_fund', 'sales_restrict', 'wbrger_obj_cd', 'policy',
       'lipper_class', 'lipper_class_name'

Index(['summary_period2', 'crsp_fundno', 'caldt', 'nav_latest',
       'nav_latest_dt', 'tna_latest', 'tna_latest_dt', 'yield', 'div_ytd',
       'cap_gains_ytd', 'nav_52w_h', 'nav_52w_h_dt', 'nav_52w_l',
       'nav_52w_l_dt', 'unrealized_app_dep', 'unrealized_app_dt', 'asset_dt',
       'per_com', 'per_pref', 'per_conv', 'per_corp', 'per_muni', 'per_govt',
       'per_oth', 'per_cash', 'per_bond', 'per_abs', 'per_mbs', 'per_eq_oth',
       'per_fi_oth', 'maturity', 'maturity_dt', 'cusip8', 'crsp_portno',
       'crsp_cl_grp', 'fund_name', 'ticker', 'ncusip', 'mgmt_name', 'mgmt_cd',
       'mgr_name', 'open_to_inv', 'retail_fund', 'inst_fund', 'm_fund',
       'index_fund_flag', 'vau_fund', 'et_flag', 'delist_cd', 'end_dt',
       'dead_flag', 'merge_fundno', 'actual_12b1', 'max_12b1', 'exp_ratio',
       'mgmt_fee', 'turn_ratio', 'fiscal_yearend', 'crsp_obj_cd', 'si_obj_cd',
       'accrual_fund', 'sales_restrict', 'wbrger_obj_cd', 'policy',
       'lipper_class', 'lipper_class_name'

In [6]:
df = pd.DataFrame({'Fund Name' : data['fund_name'], 'Fund Ticker' : data['ticker'], 
                   'Open to Investors' : data['open_to_inv'], 'Expense Ratio' : data['exp_ratio']})

In [7]:
df.isnull().sum()

Expense Ratio        73112
Fund Name             4731
Fund Ticker          60938
Open to Investors     6101
dtype: int64

Expense Ratio        73112
Fund Name             4731
Fund Ticker          60938
Open to Investors     6101
dtype: int64

In [8]:
df.dropna(inplace = True)

In [9]:
df['Active or Passive'] = df['Expense Ratio'].apply(managed)

In [10]:
df = df[df['Open to Investors'] == 'Y']

In [11]:
df.drop_duplicates(subset = ['Fund Ticker'], inplace = True)

In [12]:
df.shape

(45690, 5)

(45690, 5)

In [13]:
df.head(5)

,Expense Ratio,Fund Name,Fund Ticker,Open to Investors,Active or Passive
7,0.0242,American Pension Investors Trust: Growth Fund,APITX,Y,active
44,0.0185,UAM Funds Trust: BHM&S Total Return Bond Portf...,BHMSX,Y,active
46,0.0081,UAM Funds Trust: Chicago Asset Management Inte...,CAMBX,Y,active
55,0.0205,"Deutsche Investors Funds, Inc: Growth Opportun...",DBGPX,Y,active
58,0.0090,The DLB Fund Group: DLB Enterprise III Fund,DLBCX,Y,active


,Expense Ratio,Fund Name,Fund Ticker,Open to Investors,Active or Passive
7,0.0242,American Pension Investors Trust: Growth Fund,APITX,Y,active
44,0.0185,UAM Funds Trust: BHM&S Total Return Bond Portf...,BHMSX,Y,active
46,0.0081,UAM Funds Trust: Chicago Asset Management Inte...,CAMBX,Y,active
55,0.0205,"Deutsche Investors Funds, Inc: Growth Opportun...",DBGPX,Y,active
58,0.0090,The DLB Fund Group: DLB Enterprise III Fund,DLBCX,Y,active


In [14]:
df['Active or Passive'].value_counts()

active     34336
passive    11354
Name: Active or Passive, dtype: int64

active     34336
passive    11354
Name: Active or Passive, dtype: int64

In [15]:
df.drop(['Expense Ratio', 'Open to Investors'], axis = 1, inplace = True)

In [16]:
df.head()

,Fund Name,Fund Ticker,Active or Passive
7,American Pension Investors Trust: Growth Fund,APITX,active
44,UAM Funds Trust: BHM&S Total Return Bond Portf...,BHMSX,active
46,UAM Funds Trust: Chicago Asset Management Inte...,CAMBX,active
55,"Deutsche Investors Funds, Inc: Growth Opportun...",DBGPX,active
58,The DLB Fund Group: DLB Enterprise III Fund,DLBCX,active


,Fund Name,Fund Ticker,Active or Passive
7,American Pension Investors Trust: Growth Fund,APITX,active
44,UAM Funds Trust: BHM&S Total Return Bond Portf...,BHMSX,active
46,UAM Funds Trust: Chicago Asset Management Inte...,CAMBX,active
55,"Deutsche Investors Funds, Inc: Growth Opportun...",DBGPX,active
58,The DLB Fund Group: DLB Enterprise III Fund,DLBCX,active


In [ ]:
%%timeit

pricing_data = {}

for fund in df['Fund Ticker']:
    try:
        pricing_data[fund] = get_price(fund)
    except:
        print('Failed to get data for: ' + fund)

Failed to get data for: AADRX
Failed to get data for: AADRX
Failed to get data for: CLSAX
Failed to get data for: CLSAX
Failed to get data for: CBBCX
Failed to get data for: CBBCX


In [ ]:
returns = df_date_range(pricing_data)

In [ ]:
for k in pricing_data:
    dfs = pricing_data.get(k)
    returns[k] = np.where(returns['date'] == dfs['date'], dfs['close'], 0)

In [ ]:
returns

In [ ]:
sent['f6mret'] = sent['sp500'].shift(-26) / sent['sp500'] -1
sent['f1yret'] = sent['sp500'].shift(-52) / sent['sp500'] -1
sent['l6mret'] = sent['sp500'] / sent['sp500'].shift(26) -1
sent['l1yret'] = sent['sp500'] / sent['sp500'].shift(52) -1